In [1]:
import os

# so I don't 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"  #prevent access to GPU for inference

import pandas as pd
from pathlib import Path
from fastai.text import TextLMDataBunch as lmdb
from fastai.text.transform import Tokenizer
import pandas as pd
from pathlib import Path
from random import shuffle

from fastai.text.models import AWD_LSTM, awd_lstm_lm_config
from fastai.text import TextLMDataBunch as lmdb, load_data
from fastai.text.learner import language_model_learner
from fastai.basic_train import load_learner
from fastai.callbacks import EarlyStoppingCallback, SaveModelCallback, ReduceLROnPlateauCallback, CSVLogger


path = Path('lm_tune_data/')

In [10]:
train_df = pd.read_pickle('train_df.pkl').dropna().drop_duplicates().sample(frac=.25)
valid_df = pd.read_pickle('valid_df.pkl').dropna().drop_duplicates()

In [11]:
print(train_df.shape)
print(valid_df.shape)

(16386, 2)
(1862119, 2)


In [39]:
config_dict = awd_lstm_lm_config.update(dict(emb_sz=128))

In [41]:
awd_lstm_lm_config

{'emb_sz': 128,
 'n_hid': 1150,
 'n_layers': 3,
 'pad_token': 1,
 'qrnn': False,
 'bidir': False,
 'output_p': 0.1,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'embed_p': 0.02,
 'weight_p': 0.2,
 'tie_weights': True,
 'out_bias': True}

In [13]:
def pass_through(x):
    return x


tokenizer = Tokenizer(pre_rules=[pass_through], n_cpus=30)

path = Path('lm_tune_data/')
data_tunelm = lmdb.from_df(path=path,
                       train_df=train_df,
                       valid_df=valid_df,
                       text_cols='text',
                       tokenizer=tokenizer)

In [16]:
data_tunelm.save()

In [17]:
! ls -lah lm_tune_data_smalltest/

total 40M
drwxr-xr-x  2 root root 6.0K May 30 04:28 .
drwxr-xr-x 15 root root 6.0K May 30 04:28 ..
-rw-r--r--  1 root root  40M May 30 04:28 data_save.pkl


In [36]:
%load_ext autoreload
%autoreload 2
from lm_tune import LangModel, pass_through

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
lm = LangModel()

W&B Run: https://app.wandb.ai/github/issues_lang_model/runs/rzo9a30d
Call `%%wandb` in the cell containing your training loop to display live results.


In [38]:
lm.fit()

epoch,train_loss,valid_loss,accuracy,time
0,6.769443,6.740419,0.075352,01:20


wandb: Wandb version 0.8.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Better model found at epoch 0 with valid_loss value: 6.740419387817383.


In [52]:
doc_lengths = train_df.text.apply(lambda x: len(x.split()))

In [55]:
import numpy as np
np.median(doc_lengths)

55.0

In [61]:
tst = lm.learn.summary()

In [64]:
?? lm.learn.summary